In [1]:
import sys
import torch  
import gym
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
from torch.distributions.categorical import Categorical
import math
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from preprocess import mean, std, preprocess_input_function
from settings import train_dir, test_dir, train_push_dir, train_batch_size, test_batch_size, train_push_batch_size
from settings import base_architecture, img_size, prototype_shape, num_classes, prototype_activation_function, add_on_layers_type
from receptive_field import compute_rf_prototype
import cv2
from preference_model import construct_PrefNet, paired_cross_entropy_loss, PrefNet
from tqdm import tqdm
from settings import joint_optimizer_lrs, joint_lr_step_size

In [2]:
def unravel_index(index, shape):
    out = []
    for dim in reversed(shape):
        out.append(index % dim)
        index = index // dim
    return reversed(out)

In [3]:
'''
Use PPnet's forward pass as the policy network (actor network); what about the network for value function (critic network)?
Since there are only determinant actions, this is essentially A2C...
'''
class A3C_PPnet(nn.Module):
    def __init__(self, PPnet, preference_model, k=3, p=5, learning_rate=3e-4, dummy_reward=False):
        super(A3C_PPnet, self).__init__()
        
        
        self.PPnet = PPnet.cuda()
        #for param in self.PPnet.features.parameters():
        #    param.requires_grad = True
        self.k = k
        self.pf_model = preference_model.cuda()
        
        self.PPnet_multi = torch.nn.DataParallel(self.PPnet)
        for p in self.PPnet_multi.module.features.parameters():
            p.requires_grad = True
        for p in self.PPnet_multi.module.add_on_layers.parameters():
            p.requires_grad = True
        self.PPnet_multi.module.prototype_vectors.requires_grad = True
        for p in self.PPnet_multi.module.last_layer.parameters():
            p.requires_grad = True
        #self.critic_model = self.construct_critic().cuda()
        self.p = p
        self.critic_model = Critic().cuda()
        policy_optimizer_specs = [{'params': self.PPnet.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, 
                                  {'params': self.PPnet.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
                                  {'params': self.PPnet.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
                                  ]
        self.policy_optimizer = torch.optim.Adam(policy_optimizer_specs)
        #self.policy_optimizer = torch.optim.Adam(self.PPnet.features.parameters())
        self.critic_optimizer = torch.optim.Adam(self.critic_model.parameters())
    
        
    def get_heatmaps(self, batch_x, labels, dummy=False):
        self.PPnet_multi.eval()
        n_prototypes = self.PPnet_multi.module.num_prototypes
        prototype_shape = self.PPnet_multi.module.prototype_shape
        max_dist = prototype_shape[1] * prototype_shape[2] * prototype_shape[3]
        protoL_rf_info = self.PPnet_multi.module.proto_layer_rf_info
        
        batch_x = batch_x.cuda()
        protoL_input_torch, proto_dist_torch = self.PPnet_multi.module.push_forward(batch_x)
        #for param in self.PPnet_multi.parameters():
        #    if param.requires_grad == True:
        #        print(param.grad)
        #print(proto_dist_torch.grad)
        #proto_dist_ = np.copy(proto_dist_torch.detach().cpu().numpy())
        #print(proto_dist_torch.grad_fn)
        proto_dist_ = proto_dist_torch.view(proto_dist_torch.shape[0], proto_dist_torch.shape[1], -1)
        distances = torch.amin(proto_dist_, axis=-1)
        #distances = torch.tensor(distances)
        #print("Distances grad: ", distances.grad)
        actions = self.sample_from_distances(distances, labels)
        proto_dist = torch.clone(proto_dist_torch)
        # Move to cpu and cast to numpy here
        # proto_dist shape: (1000, 80, 7, 7)
        proto_dist = torch.transpose(proto_dist, 0, 1)
        proto_dist = proto_dist.detach().cpu().numpy()
        heatmaps = []
        joint_log_probs = []
        r = []
        for action in actions:
            img_idx, probs, j, class_identity = action[0], action[1], action[2], action[3]
            heatmaps_j = []
            r_j = []
            for i in img_idx:
                closest_patch_indices_in_distance_map_j = list(np.unravel_index(np.argmin(proto_dist[j][i],axis=None), proto_dist[j][i].shape))
                #print(closest_patch_indices_in_distance_map_j)
                closest_patch_indices_in_distance_map_j = [0] + closest_patch_indices_in_distance_map_j
                #print(closest_patch_indices_in_distance_map_j)
                closest_patch_indices_in_img = compute_rf_prototype(batch_x.size(2), closest_patch_indices_in_distance_map_j, protoL_rf_info)
                closest_patch = \
                    batch_x[i, :, closest_patch_indices_in_img[1]:closest_patch_indices_in_img[2], closest_patch_indices_in_img[3]:closest_patch_indices_in_img[4]]
                closest_patch = closest_patch.cpu().numpy()
                closest_patch = np.transpose(closest_patch, (1, 2, 0))

                original_img = batch_x[i].cpu().numpy()
                original_img = np.transpose(original_img, (1, 2, 0))
                if self.PPnet_multi.module.prototype_activation_function == 'log':
                    act_pattern = np.log((proto_dist[j][i] + 1)/(proto_dist[j][i] + self.PPnet_multi.module.epsilon))
                elif self.PPnet_multi.module.prototype_activation_function == 'linear':
                    act_pattern = max_dist - proto_dist[j][i]
                else:
                    act_pattern = prototype_activation_function_in_numpy(proto_dist[j][i])

                patch_indices = closest_patch_indices_in_img[1:5]
                
                img_size = original_img.shape[0]
                
                score = - (img_size//2 - (patch_indices[0]+patch_indices[1])//2)**2 - (img_size//2 - (patch_indices[2]+patch_indices[3])//2)**2
                
                upsampled_act_pattern = cv2.resize(act_pattern, dsize=(img_size, img_size), interpolation=cv2.INTER_CUBIC)
                rescaled_act_pattern = upsampled_act_pattern - np.amin(upsampled_act_pattern)
                rescaled_act_pattern = rescaled_act_pattern / np.amax(rescaled_act_pattern)
                heatmap = cv2.applyColorMap(np.uint8(255*rescaled_act_pattern), cv2.COLORMAP_JET)
                heatmap = np.float32(heatmap) / 255
                heatmap = heatmap[..., ::-1]
                overlayed_original_img = 0.5 * original_img + 1.0 * heatmap
                overlayed_original_img = overlayed_original_img - np.amin(overlayed_original_img)
                overlayed_original_img = overlayed_original_img / np.amax(overlayed_original_img)
                if dummy:
                    heatmaps_j.append(overlayed_original_img)
                    r_j.append(score)
                else:
                    heatmaps_j.append(overlayed_original_img)
            joint_log_prob = torch.prod(probs) * math.factorial(self.k)
            #print(joint_log_prob.grad_fn)
            heatmaps.append(heatmaps_j)
            if dummy:
                r.append(r_j)
            joint_log_probs.append(joint_log_prob)
                
        # num_prototypes * self.k heatmaps in total
        # num_prototypes probs
        #for prob in joint_log_probs:
        #    print(prob.grad_fn)
        if dummy:
            r = np.sum(np.array(r), axis=1)
            r = torch.tensor(r)
            return heatmaps, joint_log_probs, distances, r
        return heatmaps, joint_log_probs, distances
    
    def sample_from_distances(self, distances, labels):
        '''
        Takes in distances of shape (80, 1000)
        returns actions of shape (1000, ), one for each prototype
        '''
        distances = torch.clip(distances, min=1e-7, max=None)
        similarities = 1 / distances
        softmax_dist = F.softmax(similarities, dim=0)
        softmax_dist = torch.transpose(softmax_dist, 0, 1)
        # Maybe using combinatorics?
        actions = []
        for i in range(softmax_dist.shape[0]):
            class_identity = torch.argmax(self.PPnet_multi.module.prototype_class_identity[i])
            #print("Class identity: ", class_identity)
            class_dist = softmax_dist[i][labels==class_identity]
            #print("Class dist shape:", class_dist.shape)
            if len(class_dist) > self.k:
                dist = Categorical(class_dist)
                img_idx = dist.sample(sample_shape=torch.tensor([self.k]))
                probs = dist.log_prob(img_idx)
                probs = torch.exp(probs)
                actions.append([img_idx, probs, i, class_identity])
        return actions
    
    def construct_critic(self):
        critic_model = nn.Sequential(
                        nn.Linear(512 * self.k * 7 * 7, 120),
                        nn.Sigmoid(),
                        nn.Linear(120, 20),
                        nn.Sigmoid(),
                        nn.Linear(20, 1)
                        )
        return critic_model
    
    # Currently just the same architecture as the pref_net
    '''
    def critic(self, heatmaps):
        values = torch.empty(len(heatmaps))
        for i in tqdm(range(len(heatmaps))):
            #x = torch.tensor(heatmaps[i])
            x = np.concatenate(heatmaps[i], axis=1)
            x = torch.tensor(x).cuda()
            x = torch.unsqueeze(x, axis=0)
            x = torch.transpose(x, 1, 3)
            with torch.no_grad():
                x = self.pf_model.conv_features(x)
                x = torch.flatten(x, 1) # flatten all dimensions except batch
            x = self.critic_model(x)
            values[i] = x
            #print(i)
        return values
    '''
    
    # Need to vectorize
    def get_critic_inputs(self, heatmaps, dummy=False):
        critic_inputs = []
        for i in range(len(heatmaps)):
            x = np.concatenate(heatmaps[i], axis=1)
            x = torch.tensor(x).cuda()
            x = torch.unsqueeze(x, axis=0)
            x = torch.transpose(x, 1, 3)
            with torch.no_grad():
                x = self.pf_model.conv_features(x)
                x = torch.flatten(x, 1) # flatten all dimensions except batch
            critic_inputs.append(x)
        critic_inputs = torch.stack(critic_inputs, dim=0)
        critic_inputs = critic_inputs.view(critic_inputs.shape[0], -1)
        #print(critic_inputs.shape)
        return critic_inputs
        
    def get_rewards(self, heatmaps, dummy=False):
        if dummy:
            h = heatmaps
            rewards = np.empty(len(h))
            for i in range(len(h)):
                score = -np.sum(np.square(h[i][0]-h[i][1])) - np.sum(np.square(h[i][1]-h[i][2])) - np.sum(np.square(h[i][0]-h[i][2]))
            rewards[i] = score
            return torch.tensor(rewards)
        with torch.no_grad():
            rewards = torch.empty(len(heatmaps))
            for i in range(len(heatmaps)):
                pf_input = torch.tensor(heatmaps[i]).cuda()
                pf_input = pf_input.view(pf_input.shape[0]*pf_input.shape[1], pf_input.shape[2], pf_input.shape[3])
                pf_input = torch.transpose(pf_input, 0, 2)
                pf_input = torch.transpose(pf_input, 1, 2)
                pf_input = torch.unsqueeze(pf_input, axis=0)
                reward = self.pf_model(pf_input)
                rewards[i] = reward
                #print(i)
        return rewards
        
    def update_v1(self, rewards, values, probs):
        self.policy_optimizer.zero_grad()
        for prob in probs:
            prob = prob.cuda()
        rewards = rewards.cuda()
        values = values.cuda()
        #print(list(self.PPnet_multi.module.features.parameters())[0])
        policy_loss = 0
        for i in range(len(rewards)):
            policy_loss -= probs[i] * (rewards[i] - values[i])    
        policy_loss.backward(retain_graph=True)
        self.policy_optimizer.step()
        #print(list(self.PPnet_multi.module.features.parameters())[0])
        self.critic_optimizer.zero_grad()
        #print(list(self.critic_model.parameters())[0])
        critic_loss = 0
        for i in range(len(rewards)):
            critic_loss += (rewards[i] - values[i]) ** 2
        #print(critic_loss.grad_fn)
        critic_loss.backward()
        self.critic_optimizer.step()
        #print(list(self.critic_model.parameters())[0])
        
        return 
    
    def update_v2(self, rewards, values, probs):
        
        return
    
    
    def run(self, batch_x, labels):

        # action is n_prototypes * k heatmaps
        heatmaps, probs, img_distances = self.get_heatmaps(batch_x, labels)
        #print("Finished calculating heatmaps")
        critic_inputs = self.get_critic_inputs(heatmaps)
        #print("Finished getting critic inputs")
        values = self.critic_model(critic_inputs)
        #print("Finished calculating values")
        rewards = self.get_rewards(heatmaps)
        #print("Finished calculating rewards")
        self.update_v1(rewards, values, probs)
        #print("Finished updating. Done.")
        
        return rewards, values, probs
    
    def run_dummy(self, batch_x):
        heatmaps, probs, img_distances, rewards  = self.get_heatmaps(batch_x, dummy=True)
        #print("Finished calculating heatmaps")
        critic_inputs = self.get_critic_inputs(heatmaps)
        #print("Finished getting critic inputs")
        values = self.critic_model(critic_inputs)
        #print("Finished calculating values")
        #rewards = self.get_rewards(h, dummy=True)
        #print("Finished calculating rewards")
        self.update_v1(rewards, values, probs)
        
        return rewards, values, probs, heatmaps

In [4]:
class Critic(nn.Module):
    def __init__(self, k=3, learning_rate=3e-4):
        super(Critic, self).__init__()
        
        '''
        self.critic_linear1 = nn.Linear(num_inputs, hidden_size)
        self.critic_linear2 = nn.Linear(hidden_size, 1)

        self.actor_linear1 = nn.Linear(num_inputs, hidden_size)
        self.actor_linear2 = nn.Linear(hidden_size, num_actions)
        '''
        
        self.k = k
        self.fc1 = nn.Linear(512 * k * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 20)
        self.fc3 = nn.Linear(20, 1)
        
    
    def forward(self, x):

        out = torch.sigmoid(self.fc1(x))
        out = torch.sigmoid(self.fc2(out))
        out = self.fc3(out)

        return out

In [5]:
ppnet = torch.load(r'../saved_models/vgg19/004/100_7push0.7344.pth')

pf_model = construct_PrefNet("resnet18")
pf_model.load_state_dict(torch.load("./human_comparisons/pref_model_009_65+35_ep50_adam_0.0001"))
#pf_model = torch.load(r'./human_comparisons/pref_model_009_65+35_ep50_adam_0.0001_1')

<All keys matched successfully>

In [6]:
normalize = transforms.Normalize(mean=mean, std=std)

train_dataset = datasets.ImageFolder(
        train_push_dir,
        transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=100, shuffle=False,
    num_workers=4, pin_memory=False)

In [13]:
a3c = A3C_PPnet(ppnet, pf_model)

In [14]:
prob_records = np.empty((50, 1000))
reward_records = np.empty((50, 1000))
data_iter = iter(dataloader)
for _ in range(20):
    batch, labels = next(data_iter)
for i in tqdm(range(50)):
    rewards, values, probs = a3c.run(batch, labels)
    print(len(probs))
    total_reward = 0
    mse_loss = 0
    for j in range(len(probs)):
        #probs[j] = probs[j].detach().cpu().numpy()
        #rewards[j] = rewards[j].detach().cpu().numpy()
        total_reward += probs[j] * rewards[j]
        mse_loss += (rewards[j] - values[j]) ** 2
        prob_records[i][j] = probs[j].detach().cpu().numpy()
        reward_records[i][j] = rewards[j].detach().cpu().numpy()
    print("Iteration "+str(i)+" total expected reward: ", total_reward)
    print("Iteration "+str(i)+" total mse loss: ", mse_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

20


  2%|▏         | 1/50 [00:05<04:35,  5.62s/it]

Iteration 0 total expected reward:  tensor(-0.0125, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 0 total mse loss:  tensor([75.8457], device='cuda:0', grad_fn=<AddBackward0>)
20


  4%|▍         | 2/50 [00:09<03:46,  4.73s/it]

...

Iteration 47 total expected reward:  tensor(-0.0188, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 47 total mse loss:  tensor([51.5845], device='cuda:0', grad_fn=<AddBackward0>)
20


 98%|█████████▊| 49/50 [03:23<00:04,  4.13s/it]

Iteration 48 total expected reward:  tensor(-0.0181, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 48 total mse loss:  tensor([50.4070], device='cuda:0', grad_fn=<AddBackward0>)
20


100%|██████████| 50/50 [03:27<00:00,  4.15s/it]

Iteration 49 total expected reward:  tensor(-0.0204, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 49 total mse loss:  tensor([49.9347], device='cuda:0', grad_fn=<AddBackward0>)


In [12]:
np.save(r'./A3C_results/prob_records_iter10_batch20.npy', prob_records)
np.save(r'./A3C_results/reward_records_iter10_batch20.npy', reward_records)

In [39]:
prob_records_sorted = np.sort(prob_records, axis=1)
sorted_prob_index = np.argsort(prob_records, axis=1)
print(sorted_prob_index.shape)

(10, 1000)


In [42]:
prob_records[0][sorted_prob_index[0][-100:]]

array([3.1546278 , 3.33953476, 3.35639477, 3.66883135, 3.70347977,
       3.7588191 , 3.83299041, 3.83299041, 3.88497162, 3.9135766 ,
       3.95696306, 4.15255356, 4.31025791, 4.33145142, 4.36347342,
       4.46746016, 4.61634111, 4.68425179, 4.71241188, 4.9184804 ,
       4.94390678, 4.95119905, 5.11668777, 5.30823374, 5.35067034,
       5.37403584, 5.37604761, 5.42540026, 5.46156883, 5.51010084,
       5.57676554, 5.58917475, 5.62009096, 5.63833141, 5.65631962,
       5.73786974, 5.74734545, 5.75679827, 5.7850008 , 5.84051895,
       5.84336281, 5.86585045, 5.86763573, 5.88327503, 5.89575529,
       5.9021287 , 5.92162323, 5.92976141, 5.93262672, 5.94436979,
       5.95309639, 5.96435022, 5.96940517, 5.98616838, 5.98733902,
       5.99058199, 5.99097824, 5.9921999 , 5.99560738, 5.99593163,
       5.99617386, 5.99622107, 5.9969244 , 5.99782753, 5.99933243,
       5.99948931, 5.99974155, 5.99978209, 5.99981308, 5.99985504,
       5.99987555, 5.99994183, 5.99994183, 5.9999485 , 5.99995

In [48]:
reward_records[0][sorted_prob_index[0][-100:]]

array([ 2.95592165, -2.82480574, -2.66942024, -2.87655592,  2.65273333,
        2.95444059, -2.83822775, -2.83822775,  2.95611429, -2.87763977,
       -2.55860901, -2.87759066,  1.05071831,  1.99227178, -2.63136625,
        2.9560895 ,  2.95625019,  1.81558704, -2.87426138,  2.95626116,
       -2.87668371,  2.95592642,  0.56000394,  2.95593262,  2.82985115,
       -2.87542105,  2.95610237, -2.7824192 ,  2.95625544,  2.95538521,
       -0.88582468,  1.78973567,  2.84648705, -0.49043298,  1.76294279,
        2.82052374, -2.86940479,  0.74559736,  2.929389  , -2.56735229,
        2.8025589 ,  2.95449209,  2.95450878,  1.80669439,  2.95627499,
        2.39929771, -2.87768412,  2.95241928,  1.55573928, -2.63089275,
        2.95429802, -2.86952496,  2.16030598,  2.95075798,  0.91464406,
        2.95511246,  2.95502377, -2.40150547,  2.77435732,  1.88581896,
        2.95567465,  2.95625257,  1.82907784, -2.84644365,  2.37964201,
        2.9548502 ,  2.93420267,  2.95518255,  1.77334046,  2.95

In [53]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3,
        3, 3, 3, 3])

In [7]:
a3c_dummy_reward = A3C_PPnet(ppnet, pf_model)

In [8]:

prob_records_dummy = np.empty((100, 1000))
reward_records_dummy = np.empty((100, 1000))
h_records = []
data_iter = iter(dataloader)
for _ in range(20):
    batch, label = next(data_iter)
for i in tqdm(range(100)):
    rewards, values, probs, h = a3c_dummy_reward.run_dummy(batch)
    total_reward = 0
    mse_loss = 0
    
    h_records.append(h)
    for j in range(len(probs)):
        #probs[j] = probs[j].detach().cpu().numpy()
        #rewards[j] = rewards[j].detach().cpu().numpy()
        total_reward += probs[j] * rewards[j]
        mse_loss += (rewards[j] - values[j]) ** 2
        prob_records_dummy[i][j] = probs[j].detach().cpu().numpy()
        reward_records_dummy[i][j] = rewards[j].detach().cpu().numpy()
    print("Iteration "+str(i)+" total expected reward: ", total_reward)
    print("Iteration "+str(i)+" total mse loss: ", mse_loss)


  1%|          | 1/100 [00:26<42:57, 26.04s/it]

Iteration 0 total expected reward:  tensor(-1941382.1250, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 0 total mse loss:  tensor([1.0895e+10], device='cuda:0', grad_fn=<AddBackward0>)


  2%|▏         | 2/100 [00:50<41:12, 25.23s/it]

Iteration 1 total expected reward:  tensor(-571138.7500, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1 total mse loss:  tensor([1.0007e+10], device='cuda:0', grad_fn=<AddBackward0>)


  3%|▎         | 3/100 [01:15<40:37, 25.12s/it]

...

 47%|████▋     | 47/100 [19:59<22:45, 25.76s/it]

Iteration 46 total expected reward:  tensor(-7.2452, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 46 total mse loss:  tensor([1.8889e+09], device='cuda:0', grad_fn=<AddBackward0>)


 47%|████▋     | 47/100 [20:01<22:35, 25.57s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/usr/local/linux/anaconda3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
  File "/tmp/ipykernel_1654755/4072965456.py", line 8, in <cell line: 7>
  File "/tmp/ipykernel_1654755/1927103216.py", line 259, in run_dummy
  File "/tmp/ipykernel_1654755/1927103216.py", line 80, in get_heatmaps
RuntimeError: [enforce fail at CPUAllocator.cpp:65] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 602112 bytes. Error code 12 (Cannot allocate memory)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/linux/anaconda3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/usr/local/linux/anaconda3.8/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
  File "/usr/local/linux/anaconda3.8/lib/py

In [9]:
np.save(r'./A3C_results/prob_records_iter10_batch20_dummy.npy', prob_records_dummy)
np.save(r'./A3C_results/reward_records_iter10_batch20_dummy.npy', reward_records_dummy)

In [14]:
a3c.PPnet_multi.module.prototype_class_identity.shape

torch.Size([1000, 200])

(10, 1000)


In [ ]:
'''
reward_results = np.empty((50, 60))
value_loss_results = np.empty((50, 60))
for epoch in range(50):
    for i, (train_batch, label) in tqdm(enumerate(dataloader)):
    #for i in tqdm(range(100)):
        rewards, values, probs = a3c.run(train_batch)
        total_reward = 0
        mse_loss = 0
        for j in range(len(probs)):
            total_reward += probs[j] * rewards[j]
            mse_loss += (rewards[j] - values[j]) ** 2

        print("Epoch "+ str(epoch)+" Batch "+str(i)+" total expected reward: ", total_reward)
        print("Batch "+str(i)+" total mse loss: ", mse_loss)
        reward_results[epoch][i] = total_reward.detach().cpu().numpy()
        value_loss_results[epoch][i] = mse_loss.detach().cpu().numpy()
        #if i%10 == 0:
        #    torch.save(a3c.PPnet_multi, '../saved_models/iteration_'+str(i)+'_ppnet_multi_'+str(total_reward))
        #    torch.save(a3c.critic_model, '../saved_models/iteration_'+str(i)+'_critic_model_'+str(mse_loss))
        del train_batch
        for prob in probs:
            del prob
        del values
        del rewards
    avg_reward = np.mean(reward_results[epoch])
    torch.save(a3c.PPnet_multi, '../saved_models/ep_'+str(epoch)+'_ppnet_multi_'+str(avg_reward))
    torch.save(a3c.critic_model, '../saved_models/ep_'+str(epoch)+'_critic_model_'+str(avg_reward))
'''

1it [03:04, 184.88s/it]

Epoch 0 Batch 0 total expected reward:  tensor(1016.9069, device='cuda:0', grad_fn=<AddBackward0>)
Batch 0 total mse loss:  tensor([4916.5386], device='cuda:0', grad_fn=<AddBackward0>)


2it [06:13, 186.97s/it]

Epoch 0 Batch 1 total expected reward:  tensor(404.0566, device='cuda:0', grad_fn=<AddBackward0>)
Batch 1 total mse loss:  tensor([4228.8071], device='cuda:0', grad_fn=<AddBackward0>)


3it [09:24, 188.72s/it]

...

Epoch 3 Batch 37 total expected reward:  tensor(-0.0109, device='cuda:0', grad_fn=<AddBackward0>)
Batch 37 total mse loss:  tensor([3416.2495], device='cuda:0', grad_fn=<AddBackward0>)


39it [1:59:14, 184.66s/it]

Epoch 3 Batch 38 total expected reward:  tensor(-0.0118, device='cuda:0', grad_fn=<AddBackward0>)
Batch 38 total mse loss:  tensor([2894.9695], device='cuda:0', grad_fn=<AddBackward0>)


40it [2:02:17, 184.22s/it]

Epoch 3 Batch 39 total expected reward:  tensor(-0.0068, device='cuda:0', grad_fn=<AddBackward0>)
Batch 39 total mse loss:  tensor([3839.3582], device='cuda:0', grad_fn=<AddBackward0>)


NameError: name 'reward_results' is not defined

In [11]:
probs_0[:100]

[tensor(0.7500, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(5.9786, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(1.5459e-05, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(3.0292e-05, device='cuda:0', grad_fn=<MulBackward0>),
 tenso

In [12]:
probs_1[:50]

[tensor(5.9676, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(5.9924, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(3.9994, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.0007, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.3579, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(5.0587, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(2.7490e-05, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(6.0000, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(5.4254, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.5447, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.3528, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(1.0686, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(5.9681, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(3.5836, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(0.2328, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(9.6888e-06, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(1.1284e-05, device='cuda:0', grad_fn=<MulBackward0>),
 t

In [13]:
print(rewards_0[:50])
print(rewards_1[:50])

tensor([-2.7851,  0.9022,  2.8566,  2.9561,  2.5913,  1.4005,  2.8890, -0.4359,
         2.9563, -2.7223, -2.8594, -2.8131,  0.2946,  2.7151,  2.9560,  2.9544,
        -2.7062, -2.4482,  2.8733, -0.1318,  2.9515,  1.0451,  1.4682,  2.9417,
        -0.6140,  2.8225,  2.7343,  2.9544,  1.8293,  2.6736,  2.9162,  2.0157,
         2.9543, -1.0884,  2.9180, -2.7983,  2.9554,  2.1885,  1.7539,  2.9026,
         2.8248,  1.5889,  1.0819,  2.9236,  1.1265,  2.9410,  2.2216, -0.6344,
        -0.2428,  2.0169])
tensor([-2.6608,  2.1664, -2.8370, -2.2139, -1.6710,  2.9525,  1.5394,  2.9504,
         1.2801,  2.1259, -0.7744,  2.9505,  2.4044, -2.8730, -2.8742, -0.4144,
         1.0518, -2.8572,  2.9211,  2.5853,  2.1751, -0.7339,  2.9352,  2.5496,
         2.4637, -0.5362,  0.8194,  2.9032, -1.5252,  2.9526, -0.2861, -1.0242,
         2.9459,  2.2418,  2.9555, -1.8400, -0.4232,  2.9170, -1.2534,  2.3564,
         2.9544,  0.7814,  2.7956,  2.9524,  0.0044,  2.8985,  2.6865,  0.7331,
         2.88

In [ ]:
'''



def run_A3C(PPnet, preference_model, data_loader):
    
    a3c_model = A3C_PPnet(PPnet, preference_model)
    
    # TO-DO: need to figure out what optimizers we could use
    ac_optimizer = optim.Adam(PPnet.parameters(), lr=learning_rate)
    
    num_epochs = 50

    for epoch in range(num_epochs):
        for idx, (batch_x, y) in enumerate(dataloader):
            action, value, rewards = a3c_model.forward(batch_x)
            value = value.detach().numpy()
            reward = preference_model(action)

  
        # update actor critic
        
        advantage = rewards - values
        actor_loss = (-advantage).mean()
        critic_loss = 0.5 * advantage.pow(2).mean()
        ac_loss = actor_loss + critic_loss + 0.001 * entropy_term

        ac_optimizer.zero_grad()
        ac_loss.backward()
        ac_optimizer.step()

'''